In [1]:
import tensorflow as tf
import inception
import numpy as np
import os
import inception
from glob import glob
from random import shuffle
from tqdm import tqdm
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import cv2


# Transfer Learning with the pre-trained inception model

# Initialize the inception model and required initializations
inc_model = inception.Inception()
TRAIN_DIR = 'FIDS30'
MODEL_NAME = 'FIDS30_2'
LR = 1e-4

# Define a function for obtaining transfer values
def transfer_values(image_path):
    t_vals = inc_model.transfer_values(image_path)
    return t_vals

def create_train_data():
    training_data = []
    for label_p in tqdm(glob(TRAIN_DIR+'/*')):
        onehot_label = label_image(label_p.split('/')[-1])
        for img_p in tqdm(glob(label_p+'/*')):
            tr_val = transfer_values(image_path = img_p)
            training_data.append([np.array(onehot_label), np.array(tr_val)])
    shuffle(training_data)
    np.save('FIDS30_savedmodel/saved_np/FIDS30_9classes.npy',training_data)
    return

def label_image(label): # Returns the training label
    options = { 'apples':      [1, 0, 0, 0, 0, 0, 0, 0, 0],
                'bananas':     [0, 1, 0, 0, 0, 0, 0, 0, 0],
                'coconuts':    [0, 0, 1, 0, 0, 0, 0, 0, 0],
                'grapes':      [0, 0, 0, 1, 0, 0, 0, 0, 0],
                'guava':       [0, 0, 0, 0, 1, 0, 0, 0, 0],
                'mangos':      [0, 0, 0, 0, 0, 1, 0, 0, 0],
                'pineapples':  [0, 0, 0, 0, 0, 0, 1, 0, 0],
                'pomegranates':[0, 0, 0, 0, 0, 0, 0, 1, 0],
                'tomatoes':    [1, 0, 0, 0, 0, 0, 0, 0, 1],}

    return options[label]


def load_train_data():
    if glob("FIDS30_savedmodel/saved_np/FIDS30_9classes.npy"):
        train = np.load("FIDS30_savedmodel/saved_np/FIDS30_9classes.npy")
        print("Numpy file of training data exists and has been loaded")
    else:
        print("ERROR : No saved trainin data. Please run the function create_train_data()")
        exit()
    Y = np.array([i[0] for i in train])
    X = np.array([i[1] for i in train])
    return X,Y


# ======================================================================================================================
# The Fully Connected Layer (To be connected after the inception model)
# This will only have one fully connected layer and a softmax classifier

num_classes = 9
len_transfer = 2048

#Here transfer_layer is the input layer of the new network
transfer_layer = input_data(shape=[None,2048], name='input')

transfer_layer = fully_connected(transfer_layer, 1024, activation='relu')
transfer_layer = dropout(transfer_layer, 0.6)

transfer_layer = fully_connected(transfer_layer,num_classes,activation = 'softmax')
transfer_layer = regression(transfer_layer, optimizer='adam', learning_rate=LR,
                            loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(transfer_layer,tensorboard_verbose=3)

# ======================================================================================================================

def train_the_network():
    X, Y = load_train_data()

    #model.fit({'input': X}, {'targets': Y}, n_epoch=4,
              #snapshot_step=500, show_metric=True, run_id='MODEL_NAME')
    model.fit(X, Y, n_epoch=10000, batch_size=512, show_metric=True)

    model.save('FIDS30_savedmodel/'+MODEL_NAME)

    return

def test_model(image_path):
    options_num = { 0 : 'apples',
                    1 : 'bananas',
                    2 : 'coconuts',
                    3 : 'grapes',
                    4 : 'guava',
                    5 : 'mangos',
                    6 : 'pineapples',
                    7 : 'pomegranates',
                    8 : 'tomatoes',}
    
    model.load('FIDS30_savedmodel/'+MODEL_NAME)
    tr_val = transfer_values(image_path)
    tr_val = tr_val.reshape(-1,2048)
    pred = model.predict(tr_val)
    img = cv2.imread(image_path)
    print(options_num[np.argmax(pred)])
    cv2.imshow("Image",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return pred

In [59]:
pred = test_model("test_data/mango1.jpg")

INFO:tensorflow:Restoring parameters from /home/dhanush/Desktop/Internship Projects/Deep Learning Fruit Classification/TensorFlow-Tutorials/FIDS30_savedmodel/FIDS30_2


mangos


In [60]:
pred[0]

[0.12443344295024872,
 0.05848510563373566,
 0.018933037295937538,
 0.03868943080306053,
 0.02011050283908844,
 0.6382641196250916,
 0.022739024832844734,
 0.016123175621032715,
 0.06222204864025116]

In [74]:
labels =      [ 'apples',
                'bananas',
                'coconuts',
                'grapes',
                'guava',
                'mangos',
                'pineapples',
                'pomegranates',
                'tomatoes',]

In [75]:
pred_val, lab_s = zip(*sorted(zip(pred[0], labels)))

In [77]:
lab_s

('pomegranates',
 'coconuts',
 'guava',
 'pineapples',
 'grapes',
 'bananas',
 'tomatoes',
 'apples',
 'mangos')

In [81]:
for i in range(num_classes-1,-1,-1):
    print(str(lab_s[i])+' : '+str(pred_val[i]))

mangos : 0.6382641196250916
apples : 0.12443344295024872
tomatoes : 0.06222204864025116
bananas : 0.05848510563373566
grapes : 0.03868943080306053
pineapples : 0.022739024832844734
guava : 0.02011050283908844
coconuts : 0.018933037295937538
pomegranates : 0.016123175621032715
